# one-hot-encoder

In [ ]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")

col_drop = ['enrollee_id', 'city', 'experience']
col_num = ['city_development_index', 'training_hours']
col_cat = list(set(x_train.columns) - set(col_drop) - set(col_num))

x_test_id = x_test['enrollee_id']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train['target']

# 명목형
from sklearn.preprocessing import OneHotEncoder

X = pd.concat([x_train, x_test])

ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(X[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())


x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

y_train = y_train.astype('int')


from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_tr_pred = model.predict(x_tr)
y_val_pred = model.predict(x_val)

print('train ac : ', accuracy_score(y_tr_pred, y_tr))
print('val ac : ', accuracy_score(y_val_pred, y_val))


from sklearn.metrics import roc_auc_score

y_val_pred_probd = model.predict_proba(x_val)[:,1]
print(roc_auc_score(y_val, y_val_pred_probd))

y_pred = model.predict(x_test_fin)
result = pd.DataFrame({'enrollee_id' : x_test_id, 'target' : y_pred})
result.to_csv('0030.csv', index = False)

# 학률
# y_pred = model.predict_proba(x_test_dum)[:, 1]
# result = pd.DataFrame({'enrollee_id' : x_test_id, 'target' : y_pred})
# result.to_csv('0030.csv', index = False)

train ac :  0.9963285910968334
val ac :  0.7673982869379015
0.7594579508327265


In [ ]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/x_test.csv")

col_drop = ['ID']
col_num = ['age', 'bmi', 'children']
col_cat = ['sex', 'smoker', 'region']
test_id = x_test['ID']

x_train = x_train.drop(col_drop, axis = 1)
x_test = x_test.drop(col_drop, axis = 1)
y_train = y_train.drop(col_drop, axis = 1)

# ohe
x = pd.concat([x_train, x_test])

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[col_cat])

x_train_res = ohe.transform(x_train[col_cat])
x_test_res = ohe.transform(x_test[col_cat])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

# split
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3)

#scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

# model
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(x_tr, y_tr)

from sklearn.metrics import mean_squared_error , mean_absolute_error , mean_absolute_percentage_error ,r2_score
y_tr_pred = model.predict(x_tr)
y_val_pred = model.predict(x_val)

print('tr mse : ', mean_squared_error(y_tr, y_tr_pred))
print('val mse : ', mean_squared_error(y_val, y_val_pred))

print('tr rmse : ', mean_squared_error(y_tr, y_tr_pred, squared = False))
print('val rmse : ', mean_squared_error(y_val, y_val_pred, squared = False))

print('tr mae : ', mean_absolute_error(y_tr, y_tr_pred))
print('val mae : ', mean_absolute_error(y_val, y_val_pred))

print('tr mspe : ', mean_absolute_percentage_error(y_tr, y_tr_pred))
print('val mspe : ', mean_absolute_percentage_error(y_val, y_val_pred))

print('tr r2 : ', mean_squared_error(y_tr, y_tr_pred))
print('val r2 : ', mean_squared_error(y_val, y_val_pred))

print('tr mse : ', r2_score(y_tr, y_tr_pred))
print('val mse : ', r2_score(y_val, y_val_pred))

y_pred = model.predict(x_test_fin)
result = pd.DataFrame({'ID' : test_id, 'charges' : y_pred})
result.to_csv('00300.csv', index = False)

# col이 다른 dummies

In [ ]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")



# col drop, null, 이상치, object
drop_col = ['carID']
col_num = ['year', 'mileage', 'tax', 'mpg', 'engineSize']
col_cat = ['brand', 'model', 'transmission', 'mileage', 'fuelType']
y_col = ['price']
x_test_copy = x_test.copy()

x_train = x_train.drop(drop_col, axis = 1)
x_test = x_test.drop(drop_col, axis = 1)
y_train = y_train[y_col]

# object -> num
dum_col = ['brand', 'transmission', 'fuelType']

X_train_dum = pd.get_dummies(x_train[dum_col])
X_test_dum = pd.get_dummies(x_test[dum_col])

train_cols = set(X_train_dum.columns)
test_cols = set(X_test_dum.columns)

missing_in_test = train_cols - test_cols
for col in missing_in_test:
    X_test_dum[col] = 0

missing_in_train = test_cols - train_cols
for col in missing_in_train:
    X_train_dum[col] = 0

X_test_dum = X_test_dum[X_train_dum.columns]


x_train_fin = pd.concat([X_train_dum, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([X_test_dum, x_test[col_num]], axis = 1)


# split, scaler,model
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3)

# scaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test_fin[col_num])

# model
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_tr, y_tr)

from sklearn.metrics import r2_score
y_val_pred = model.predict(x_val)
print(r2_score(y_val, y_val_pred))

# 평가
from sklearn.metrics import mean_squared_error
y_pred = model.predict(x_test_fin)
pd.DataFrame({'carID' : x_test_copy['carID'],'price' : y_pred}).to_csv('00030.csv', index = False)

# 결측치 처리

In [ ]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e4_p2_test.csv')

## 명목형 nan -> 가장 많이 나온 값을 넣기(.value_counts().sort_index(ascending = False).index[0])
## 수치형 nan -> 평균 넣기
## train nan처리
train['Ever_Married'] = train['Ever_Married'].fillna(train['Ever_Married'].value_counts().sort_values(ascending = False).index[0])
train['Graduated'] = train['Graduated'].fillna(train['Graduated'].value_counts().sort_values(ascending = False).index[0])
train['Profession'] = train['Profession'].fillna(train['Profession'].value_counts().sort_values(ascending = False).index[0])
train['Work_Experience'] = train['Work_Experience'].fillna(train['Work_Experience'].mean())
train['Family_Size'] = train['Family_Size'].fillna(train['Family_Size'].mean())
train['Var_1'] = train['Var_1'].fillna(train['Var_1'].value_counts().sort_values(ascending = False).index[0])

## test nan 처리
test['Ever_Married'] = test['Ever_Married'].fillna(test['Ever_Married'].value_counts().sort_values(ascending = False).index[0])
test['Graduated'] = test['Graduated'].fillna(test['Graduated'].value_counts().sort_values(ascending = False).index[0])
test['Profession'] = test['Profession'].fillna(test['Profession'].value_counts().sort_values(ascending = False).index[0])
test['Work_Experience'] = test['Work_Experience'].fillna(test['Work_Experience'].mean())
test['Family_Size'] = test['Family_Size'].fillna(test['Family_Size'].mean())
test['Var_1'] = test['Var_1'].fillna(test['Var_1'].value_counts().sort_values(ascending = False).index[0])


col_drop = ['ID']
col_num = ['Age', 'Work_Experience', 'Family_Size']
col_cat = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
col_y = ['Segmentation']

train = train.drop(col_drop, axis = 1)
test = test.drop(col_drop, axis = 1)
x_train = train.loc[:, 'Gender' : 'Var_1']
y_train = train[col_y]
x_test = test

# 명목형
ohe_col = ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']
from sklearn.preprocessing import OneHotEncoder
x = pd.concat([x_train, x_test])

ohe = OneHotEncoder(handle_unknown = 'ignore')
ohe.fit(x[ohe_col])

x_train_res = ohe.transform(x_train[ohe_col])
x_test_res = ohe.transform(x_test[ohe_col])

x_train_ohe = pd.DataFrame(x_train_res.todense(), columns = ohe.get_feature_names_out())
x_test_ohe = pd.DataFrame(x_test_res.todense(), columns = ohe.get_feature_names_out())

x_train_fin = pd.concat([x_train_ohe, x_train[col_num]], axis = 1)
x_test_fin = pd.concat([x_test_ohe, x_test[col_num]], axis = 1)

# split
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train_fin, y_train.values.ravel(), test_size = 0.3, stratify = y_train.values.ravel())

# scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_tr[col_num])

x_tr[col_num] = scaler.transform(x_tr[col_num])
x_val[col_num] = scaler.transform(x_val[col_num])
x_test_fin[col_num] = scaler.transform(x_test[col_num])

#model
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
y_pred_train = model.predict(x_tr)
print('train as : ', accuracy_score(y_pred_train, y_tr))

y_pred_val = model.predict(x_val)
print('val as : ', accuracy_score(y_pred_val, y_val))

from sklearn.metrics import f1_score
y_pred_train = model.predict(x_tr)
print('train f1 : ', f1_score(y_tr, y_pred_train, average = 'macro'))

y_pred_val = model.predict(x_val)
print('val f1 : ', f1_score(y_val, y_pred_val, average = 'macro'))

# 제출
y_pred = model.predict(x_test_fin)
result = pd.DataFrame({'index' : x_test.index, 'Segmentation' : y_pred})
result.to_csv('00300.csv', index = False)